Kyle Hix Question 3
GenAI was used only to produce code
 - I have not coded in python prior to this course and needed assistance to complete the quiz. Providing full disclosure as per syllabus.

A)

In [ ]:
import pandas as pd

# Load the CSV file
file_path = "/content/Redacted-Department _ CAD Reconciliation.csv"  # replace with your actual file

# Read the CSV into a DataFrame
df = pd.read_csv(file_path)

# Count number of rows
row_count = len(df)

print(f"Number of rows: {row_count}")

a.1)The range is 2200 cases.

In [ ]:
missing_percent = df.isnull().mean() * 100

print("Percentage of missing data per column:")
print(missing_percent)

a.2) Percentage of missing data per column:
XREF ID                   0.000000
DISPATCH UNIT             0.000000
DISPATCH CREATED DATE     0.000000
INCIDENT NUMBER           0.000000
1ST UNIT ON SCENE        19.454545
ALARM DATE TIME           1.409091
CALL COMPLETE             1.409091
SHIFT                     3.136364

a.3) One issue with the data is that the missing entries can't be filled in. For instance, the column 1st Unit on Scene. There is no mean to fill in the void. A solution would be to list all dispatch units, list the mode, or to remove that column entirely.

In [ ]:

if "ID" not in df.columns:
    df.insert(0, "ID", range(1, len(df) + 1))

for col in df.select_dtypes(include=["float64", "int64"]).columns:
    df[col].fillna(df[col].median(), inplace=True)

for col in df.select_dtypes(include=["object"]).columns:
    if df[col].isnull().any():
        df[col].fillna(df[col].mode()[0], inplace=True)

df.drop_duplicates(inplace=True)

missing_percent = df.isnull().mean() * 100
print("Remaining missing data (%):")
print(missing_percent)

df.to_csv("cleaned_file.csv", index=False)


a.3) The empty categorical data was filled in with the mode, while numerical data was filled in with the mean.

B.

In [ ]:
df['DISPATCH CREATED DATE'] = pd.to_datetime(df['DISPATCH CREATED DATE'], errors="coerce", infer_datetime_format=True)
df['CALL COMPLETE'] = pd.to_datetime(df['CALL COMPLETE'],  errors="coerce", infer_datetime_format=True)

df['duration'] = df['CALL COMPLETE'] - df['DISPATCH CREATED DATE']

average_duration = df['duration'].mean()

print("Average time between start and stop:", average_duration)

Average resolution time: 90 days 07:23:52.39

In [ ]:
df['unit_count'] = df['DISPATCH UNIT'].apply(
    lambda x: len(str(x).split(',')) if pd.notnull(x) else 0
)

avg_units_per_case = df.groupby('INCIDENT NUMBER')['unit_count'].mean().mean()

print("Average dispatch units per case:", avg_units_per_case)

b.2) On average, 1.44 units are sent. So between one and two units.

In [ ]:
most_common_shift = df['SHIFT'].value_counts().idxmax()
shift_counts = df['SHIFT'].value_counts()

print("Most common shift:", most_common_shift)
print("\nShift frequency counts:")
print(shift_counts)

b.3) Shift A appears the most.

In [ ]:
df['ALARM DATE TIME'] = pd.to_datetime(df['ALARM DATE TIME'], errors="coerce", infer_datetime_format=True)

df['DayOfWeek'] = df['ALARM DATE TIME'].dt.day_name()
df['Hour'] = df['ALARM DATE TIME'].dt.hour

matrix = pd.pivot_table(
    df,
    index='Hour',
    columns='DayOfWeek',
    values='INCIDENT NUMBER',
    aggfunc='count',
    fill_value=0
)

days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
matrix = matrix.reindex(columns=days_order)

matrix['Total'] = matrix.sum(axis=1)
matrix.loc['Total'] = matrix.sum(axis=0)


print(matrix)

C.

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score


df['ALARM DATE TIME'] = pd.to_datetime(df['ALARM DATE TIME'], errors="coerce")


df['Hour'] = df['ALARM DATE TIME'].dt.hour
df['DayOfWeek'] = df['ALARM DATE TIME'].dt.dayofweek  
df['unit_count'] = df['DISPATCH UNIT'].apply(lambda x: len(str(x).split(',')) if pd.notnull(x) else 0)


X = df[['Hour', 'DayOfWeek', 'unit_count']]


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


kmeans = KMeans(n_clusters=3, random_state=42)
kmeans_labels = kmeans.fit_predict(X_scaled)


agg = AgglomerativeClustering(n_clusters=3)
agg_labels = agg.fit_predict(X_scaled)


kmeans_sil = silhouette_score(X_scaled, kmeans_labels)
kmeans_dbi = davies_bouldin_score(X_scaled, kmeans_labels)

agg_sil = silhouette_score(X_scaled, agg_labels)
agg_dbi = davies_bouldin_score(X_scaled, agg_labels)

print("KMeans -> Silhouette:", kmeans_sil, " DBI:", kmeans_dbi)
print("Agglomerative -> Silhouette:", agg_sil, " DBI:", agg_dbi)


if kmeans_sil > agg_sil and kmeans_dbi < agg_dbi:
    print("KMeans performs better overall.")
else:
    print("Agglomerative performs better overall.")


C.1) KMeans performs better overall